In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import os
import pickle
from collections import OrderedDict

import nept

from loading_data import get_data
from utils_maze import get_xyedges, find_zones, expand_line, speed_threshold
from plot_decode import load_decoded, get_zone_proportion, get_proportion_normalized
from utils_plotting import plot_decoded_compare, plot_intersects, plot_zone

In [ ]:
import info.r066d3 as info

In [ ]:
# data_filepath = 'C:/Users/Emily/code/emi_shortcut/cache/data'
# pickle_filepath = 'C:/Users/Emily/code/emi_shortcut/cache/pickled'

data_filepath = 'E:/code/emi_shortcut/cache/data'
pickle_filepath = 'E:/code/emi_shortcut/cache/pickled'

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = get_xyedges(position)

In [ ]:
neurons_filename = info.session_id + '_neurons.pkl'
pickled_neurons = os.path.join(pickle_filepath, neurons_filename)
with open(pickled_neurons, 'rb') as fileobj:
    neurons = pickle.load(fileobj)

In [ ]:
experiment_time = 'phase3'
decoded_filename = info.session_id + '_decode-' + experiment_time + '.pkl'
pickled_decoded = os.path.join(pickle_filepath, decoded_filename)
with open(pickled_decoded, 'rb') as fileobj:
    decoded = pickle.load(fileobj)

In [ ]:
infos = [info, info]

In [ ]:
experiment_times = ['phase1', 'phase2', 'phase3']
decodes = []
for this_info in infos:
    decodes.append(load_decoded(this_info, experiment_times, pickle_filepath, get_proportion_normalized))

In [ ]:
plot_decoded_compare(decodes)

In [ ]:
def compare_lengths(infos, combined_zones, timebin=0.025):
    """Compare position normalized by time spent in zone.

    Parameters
    ----------
    infos: list of modules
    combined_zones: list of OrderedDict of dict of lists
        With experiment_time as keys and inner dict
        has u, shortcut, novel as keys.

    Returns
    -------
    by_track_length : list
        Of OrderedDict with experiment_times as keys,
        each a dict with u, shortcut, novel as keys.

    """
    lengths = dict(u=[], shortcut=[], novel=[])
    for info in infos:
        lengths['u'].append(LineString(info.u_trajectory).length)
        lengths['shortcut'].append(LineString(info.shortcut_trajectory).length)
        lengths['novel'].append(LineString(info.novel_trajectory).length)

    by_track_length = []

    for i, session in enumerate(combined_zones):
        normalized = OrderedDict()
        for exp in combined_zones[0].keys():
            normalized[exp] = dict()

        for exp in session.keys():
            for trajectory in session[exp].keys():
                # normalized[exp][trajectory] = (session[exp][trajectory].n_samples * timebin) / lengths[trajectory][i]
                normalized[exp][trajectory] = (session[exp][trajectory].n_samples) / lengths[trajectory][i]

        by_track_length.append(normalized)

    return by_track_length

In [ ]:
def get_proportion_normalized(info, decoded, experiment_time):
    """Finds decoded proportions normalized by track length
    
    Parameters
    ----------
    info: module
    decoded: nept.Position
    experiment_time: str
    
    Returns
    -------
    normalized_proportions: dict
        With u, shortcut, novel as keys
    
    """
    print('Working on', info.session_id)
    if experiment_time not in ['prerecord', 'phase1', 'pauseA', 'phase2', 'pauseB', 'phase3', 'postrecord']:
        raise ValueError("experiment time is not recognized as a shortcut experiment time.")
        
    length = dict()
    length['u'] = LineString(info.u_trajectory).length
    length['shortcut'] = LineString(info.shortcut_trajectory).length
    length['novel'] = LineString(info.novel_trajectory).length
    
    n_total = 0
    for zone in ['u', 'shortcut', 'novel']:
        n_total += decoded['zones'][zone].n_samples / length[zone]
    n_total = np.maximum(n_total, 1.0)

    decoded_proportions = dict()
    for zone in ['u', 'shortcut', 'novel']:
        decoded_proportions[zone] = (decoded['zones'][zone].n_samples / length[zone]) / n_total

    return decoded_proportions

In [ ]:
get_proportion_normalized(info, decoded, 'phase1')

In [ ]:
45.5+30+23.9


In [ ]:
experiment_time = 'phase3'
min_swr=3
min_sequence=3
speed_limit=0.4
min_epochs=3
shuffle_id=False

In [ ]:
print('decoding:', info.session_id, experiment_time)

track_times = ['phase1', 'phase2', 'phase3', 'tracks']
pedestal_times = ['pauseA', 'pauseB', 'prerecord', 'postrecord']

events, position, all_spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)

exp_start = info.task_times[experiment_time].start
exp_stop = info.task_times[experiment_time].stop

sliced_spikes = neurons.time_slice(exp_start, exp_stop)

if experiment_time in track_times:
    run_position = speed_threshold(position, speed_limit=speed_limit)
else:
    run_position = position

exp_position = run_position.time_slice(exp_start, exp_stop)

if shuffle_id:
    random.shuffle(neurons.tuning_curves)

In [ ]:
if experiment_time in track_times:
    epochs_interest = nept.Epoch(np.hstack([exp_start, exp_stop]))
elif experiment_time in pedestal_times:
    sliced_lfp = lfp.time_slice(exp_start, exp_stop)

    z_thresh = 3.0
    power_thresh = 5.0
    merge_thresh = 0.02
    min_length = 0.05
    swrs = nept.detect_swr_hilbert(sliced_lfp, fs=info.fs, thresh=(140.0, 250.0), z_thresh=z_thresh,
                                  power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)

    print('sharp-wave ripples, total:', swrs.n_epochs)

    min_involved = 4
    epochs_interest = nept.find_multi_in_epochs(sliced_spikes, swrs, min_involved=min_involved)

    print('sharp-wave ripples, min', min_involved, 'neurons :', epochs_interest.n_epochs)

    if epochs_interest.n_epochs < min_swr:
        epochs_interest = nept.Epoch(np.array([[], []]))

    print('sharp-wave ripples, used :', epochs_interest.n_epochs)
    print('sharp-wave ripples, mean durations: ', np.mean(epochs_interest.durations))
else:
    raise ValueError("unrecognized experimental phase. Must be in ['prerecord', 'phase1', 'pauseA', 'phase2', "
                     "'pauseB', phase3', 'postrecord'].")

In [ ]:
decode_spikes = []
decode_tuning_curves = []
for spiketrain, tuning_curve in zip(sliced_spikes, neurons.tuning_curves):
    if len(spiketrain.time) > 0:
        decode_spikes.append(spiketrain)
        decode_tuning_curves.append(tuning_curve)
decode_tuning_curves = np.array(decode_tuning_curves)

In [ ]:
window_size = 0.020
window_advance = 0.005
# window_size = 0.025
# window_advance = 0.025
time_edges = nept.get_edges(exp_position, window_advance, lastbin=True)
counts = nept.bin_spikes(decode_spikes, exp_position, window_size, 
                         window_advance, gaussian_std=None, normalized=True)

In [ ]:
tc_shape = decode_tuning_curves.shape
decoding_tc = decode_tuning_curves.reshape(tc_shape[0], tc_shape[1] * tc_shape[2])

likelihood = nept.bayesian_prob(counts, decoding_tc, window_advance)

In [ ]:
# pp = plt.pcolormesh(counts.data.T, cmap='pink_r')
# plt.colorbar(pp)
# plt.axis('off')
# plt.show()

In [ ]:
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = nept.cartesian(xcenters, ycenters)

decoded = nept.decode_location(likelihood, xy_centers, time_edges)
nan_idx = np.logical_and(np.isnan(decoded.x), np.isnan(decoded.y))
decoded = decoded[~nan_idx]

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(decoded.x, decoded.y, bins=xx.shape)

cmap = plt.cm.get_cmap('bone_r', 25)
pp = plt.pcolormesh(yy, xx, histogram, cmap=cmap)
plt.colorbar(pp)
# plt.axis('off')
plt.show()

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(decoded.x, decoded.y, bins=xx.shape)

cmap = plt.cm.get_cmap('bone_r', 25)
pp = plt.pcolormesh(yy, xx, histogram, cmap=cmap)
plt.colorbar(pp)
# plt.axis('off')
plt.show()

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(exp_position.x, exp_position.y, bins=xx.shape)

cmap = plt.cm.get_cmap('bone_r', 25)
pp = plt.pcolormesh(yy, xx, histogram, cmap=cmap)
plt.colorbar(pp)
# plt.axis('off')
plt.show()

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(decoded.x, decoded.y, bins=xx.shape)

cmap = plt.cm.get_cmap('bone_r', 25)
pp = plt.pcolormesh(yy, xx, histogram, cmap=cmap)
plt.colorbar(pp)
# plt.axis('off')
plt.show()

In [ ]:
spikes = np.hstack((np.arange(0, 10, 1.4), np.arange(0.2, 5, 0.3)))
spikes = [nept.SpikeTrain(np.sort(spikes), 'test')]

position = nept.Position(np.array([0, 2, 4, 6, 8, 10]), np.array([0, 2, 4, 6, 8, 10]))
counts = nept.bin_spikes(spikes, position, window_size=2.,
                             window_advance=2., gaussian_std=None, normalized=False)

In [ ]:
counts.data

In [ ]:
import nept
import numpy as np

In [ ]:
spikes = [nept.SpikeTrain([0.8, 1.1, 1.2, 1.2, 2.1, 3.1])]
position = nept.Position(np.array([1., 6.]), np.array([0., 4.]))

counts = nept.bin_spikes(spikes, position, window_size=2.,
                         window_advance=0.5, gaussian_std=None)

# assert np.allclose(counts.data, np.array([[0.25], [1.], [1.], [1.25], [1.], [0.5], [0.5]]))

In [ ]:
counts.data

In [ ]:
array([[5.53572467e-01],
       [8.87713270e-01],
       [1.05429341e+00],
       [1.02404730e+00],
       [8.04477482e-01],
       [5.62670198e-01],
       [3.94625765e-01],
       [2.62864946e-01],
       [1.41733188e-01],
       [4.50338310e-02],
       [6.53431203e-03],
       [3.87072740e-04],
       [8.98294933e-06],
       [8.01892607e-08],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00]])

In [ ]:
import scipy.signal as signal
n_gaussian_std = 5
gaussian_std = 0.005

n_points = n_gaussian_std * gaussian_std * 2 / window_advance
n_points = max(n_points, 1.0)
if n_points % 2 == 0:
    n_points += 1
gaussian_filter = signal.gaussian(n_points, gaussian_std / window_advance)
gaussian_filter /= np.sum(gaussian_filter)

smoothed_spikes = []
for spiketrain in decode_spikes:
    if len(spiketrain.time) > 0:
        smoothed_spikes.append(nept.SpikeTrain(np.convolve(spiketrain.time,
                                                           gaussian_filter, mode='same'),
                                               spiketrain.label))

In [ ]:
smoothed_spikes